In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Placement Prediction Using XGB Classifier and Randomized Search CV 

# Importing Necessary Libraries

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import sklearn.metrics as metrics
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import seaborn as sns

# Reading Data

In [ ]:
data = pd.read_csv('/kaggle/input/engineering-placements-prediction/collegePlace.csv')

In [ ]:
data.head()

# Preprocessing Data

## Checking for Null values

In [ ]:
data.isnull().sum()

## Checking for datatypes

In [ ]:
data.info()

## Gender and Stream are object type

## Encoding Category Data

In [ ]:
data['Gender'].unique()

In [ ]:
data['Stream'].unique()

In [ ]:
en = LabelEncoder()
catCols = ['Gender','Stream']
for cols in catCols:
    data[cols] = en.fit_transform(data[cols])

In [ ]:
data.head()

In [ ]:
data.info()

# Splitting Data Into X and Y

In [ ]:
Y = data["PlacedOrNot"]
X = data.drop("PlacedOrNot",axis = 1)

In [ ]:
Y=pd.DataFrame(Y)

In [ ]:
X.head()

In [ ]:
Y.head()

# Splitting Test Train Data

In [ ]:
# Break off validation set from training data
X_train_full, X_valid_full, Y_train, Y_valid = train_test_split(X, Y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# XGB Classifier

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score, roc_auc_score
# Define the model

    
    # defining the model
clf = XGBClassifier(learning_rate=0.09, 
                    n_estimators=100, 
                    use_label_encoder=False,
                    random_state=42)
    
clf.fit(X_train_full, Y_train, eval_metric='logloss')
predictions = clf.predict(X_valid_full) 
print("accuracy_score: " + str(accuracy_score(Y_valid, predictions)))

In [ ]:
print("f1_score: " + str(f1_score(Y_valid, predictions)))

In [ ]:
predictions


In [ ]:
X.head(18)

In [ ]:
data['Stream'].unique()

# Using Randomized Search CV

In [ ]:
#Randomized Search CV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# lr
learning_rate = [ float(x) for x in [0.9, 0.1, 0.09, 0.01]]

In [ ]:
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'learning_rate': learning_rate,
               }

print(random_grid)


In [ ]:
clf2 = XGBClassifier()

In [ ]:
model = RandomizedSearchCV(estimator = clf2, param_distributions = random_grid, n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = 1)

In [ ]:
model.fit(X_train_full, Y_train, eval_metric='logloss')

In [ ]:
model.best_params_

In [ ]:
predictions2 = model.predict(X_valid_full) 
print("accuracy_score: " + str(accuracy_score(Y_valid, predictions2)))

# Making Predictions

## Uncomment the section for realtime input

In [ ]:
"""print("Enter Age : ")
age=int(input())
print("\nEnter Gender(M/F) : ")
gen=input()
if(gen=='M'):
    gender=1
    gender=int(gender)
elif(gen=='F'):
    gender=0
    gender=int(gender)
print("\nEnter Stream(ECE/CS/IT/MEC/ELE/CIV) : ")
stre=input()
if(stre=='ECE'):
    stream=3
elif(stre=='CS'):
    stream=1
elif(stre=='IT'):
    stream=4
elif(stre=='MEC'):
    stream=5
elif(stre=='ELE'):
    stream=2
elif(stre=='CIV'):
    stream=0
stream=int(stream)
print("\nEnter Number of Internships Done : ")
intern = int(input())
print("\nEnter CGPA : ")
cgpa=int(input())
print("\nEnter Hostel Accommodation : ")
hostel=int(input())
print("\nEnter number of backlogs : ")
back=int(input())"""
age=20
gender=1
stream=1
intern=3
cgpa=8
hostel=1
back=1
pred = clf.predict(np.array([[age,gender,stream,intern,cgpa,hostel,back]]))
if(pred==1):
    print("\nYou have high chances of getting placed")
else:
    print("\nYou have high chances of getting placed. Work more Hard!")